# Similarity and Distance

피처 사이의 관계는 벡터 사이의 관계로 환원될 수 있다. 벡터 사이의 관계는 다시 서로 얼마나 비슷한가, 반대로 얼마나 떨어져 있는가라는 물음으로 환원될 수 있다. 

따라서 벡터 사이의 유사도()와 거리()를 구해야 할 필요가 생겨난다. 

유사도와 거리는 반대 대체로 반대 개념이지만, 각각의 특징이 있다. 

* 유사도
  - "얼마나 비슷한가"에 대한 값이다.
  - 주로 0에서 1 사이의 값을 가지며, 0은 전혀 같지 않은 상태(벡터에서의 직교와 같은 위상), 1은 완전히 같은 상태.
  
* 거리
  - "얼마나 떨어져 있는가"에 대한 값이다.
  - 주로 0에서 무한대의 값을 가진다. 0은 완전히 같은 상태.


유사도를 구하는 방법은 매우 다양하다. 그 가운데 가장 보편적으로 사용하는 방법은 코사이 유사도(cosine similarity)이다. 

코사인 유사도는 두 벡터가 이루는 내각의 코사인 값을 의미한다. 따라서 직교하면 0이 되고, 완전히 일치하면 1이 된다. 

코사인 유사도는 두 벡터의 내적을 이용하여 간단하게 구할 수 있기 때문에 계산에 있어서도 유리하다. 

거리를 구하는 방법 역시 다양하다. 그 가운데 가장 보편적으로 사용하는 방법은 유클리디안 거리(Euclidian distance)이다. 

유클리디안 거리는 두 벡터 사이의 직선 거리를 의미한다. 따라서 완전히 같은 곳에 있으면 0이 되고, 거리가 늘어남에 따라 크기가 커진다. 


여기에서는 word vector를 구한 뒤에, word vector 사이의 코사인 유사도를 통해 가장 비슷한 단어들을 연쇄적으로 추출하는 작업을 수행해 보고자 한다. 

## Term Vector

In [1]:
from collections import Counter
import numpy as np

In [2]:
corpus_path = "../data/formulas.txt"
corpus_ = open( corpus_path, 'r', encoding='utf-8' ).read()
header, corpus_raw = corpus_.split("***")
corpus_raw = corpus_raw.strip()
corpus = [ line.strip() for line in corpus_raw.split("\n") ]
corpus_tokenized = [ line.split() for line in corpus ]

print( "# Corpus Description" )
print( header.strip() )
print()
print( "# Corpus Size: ", len(corpus) )

# Corpus Description
- 출처 : 한국전통지식포탈(www.koreantk.com) > 전통의료 > 처방
- 특징 : 본초 구성만 추출한 데이터
- 데이터 생성일 : 2016.01.16

# Corpus Size:  19162


### Count based embedding : 1st Order Vector ( TF, TFIDF Vector )

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from time import time

In [4]:
min_df = 6

In [5]:
# Build TF Matrix and TF-IDF Matrix

t0 = time()
tf_vectorizer = CountVectorizer( min_df=min_df )
tf = tf_vectorizer.fit_transform( corpus )
tf_feature_names = tf_vectorizer.get_feature_names()
print( "TF Matrix done in {:03f}s.".format(time() - t0) )

t0 = time()
tfidf_vectorizer = TfidfVectorizer( min_df=min_df )
tfidf = tfidf_vectorizer.fit_transform( corpus )
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print( "TF-IDF Matrix done in {:03f}s.".format(time() - t0) )

doc_size, feature_size = tf.shape
print( "# 처방 개수: ", doc_size)
print( "# 본초 개수: ", feature_size, " ({}회 이상 사용된 본초)".format( min_df ) ) 

term_vector_via_tf = tf.T
term_vector_via_tfidf = tfidf.T

freq_per_term = np.sum( tf, axis=0 ).flatten().tolist()[0]
size_per_doc = np.sum( tf, axis=1 ).flatten().tolist()[0]

TF Matrix done in 0.125026s.
TF-IDF Matrix done in 0.128977s.
# 처방 개수:  19162
# 본초 개수:  916  (6회 이상 사용된 본초)


### Count based embedding : 2nd Order Vector ( Co-word, t-score Vector )

In [6]:
co_word = ( tf.T * tf )
term_vector_via_coword = co_word
print( "{}개 본초에 대해 {} 길이의 벡터 생성".format( *co_word.shape ) )

916개 본초에 대해 916 길이의 벡터 생성


In [7]:
# Observed Value
observed_v = co_word.toarray()

# Expected Value
margin_x = np.full( co_word.shape, freq_per_term )
margin_y = margin_x.T
expected_v = np.divide(  np.multiply( margin_x, margin_y ),  sum( freq_per_term )   )

# T-score with addone smoothing
observed_v_add1 = np.add( observed_v, 1 )
t_score = np.divide( np.subtract( observed_v, expected_v ) , np.sqrt( observed_v_add1 ) )

from scipy import sparse
term_vector_via_tscore = sparse.csc_matrix( t_score )
print( "{}개 본초에 대해 {} 길이의 벡터 생성".format( *term_vector_via_tscore.shape ) )

916개 본초에 대해 916 길이의 벡터 생성


### Prediction based embedding

In [8]:
import gensim

vec_size = 64
pochs = 32
max_formula_length = max( size_per_doc ) 
print( "Window size: ", max_formula_length )

w2v = gensim.models.Word2Vec( size=vec_size, window=max_formula_length, min_count=min_df, workers=10 )
w2v_feature_names = w2v.build_vocab( corpus_tokenized )
w2v.train( corpus_tokenized, total_examples=len( corpus_tokenized ), epochs=pochs)


C:\Users\Junho\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Window size:  55


(2816139, 4486336)

In [9]:
def w2v_most_similar( test_herb, n=4 ):
    return w2v.wv.most_similar( [ test_herb ], topn=n )

w2v_most_similar( "감초" )

[('자감초', 0.7659971117973328),
 ('분초', 0.5264178514480591),
 ('건갈', 0.39061635732650757),
 ('천궁', 0.39024609327316284)]

## 결과 비교


### Neighbors

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

class MostElem:
    
    def __init__(self, feature_list, embedding_matrix, max_n=4):
        self.feature_list = feature_list
        self.embedding_matrix = embedding_matrix
        self.max_n = max_n
    
    def most_similar( self, term ):
        i = self.feature_list.index( term )
        sims = cosine_similarity( self.embedding_matrix[i,:], self.embedding_matrix )
        most_sim_idx = sims.argsort()[0][-self.max_n-1:-1].tolist()
        most_sim_idx.reverse()
        return [ ( self.feature_list[i], sims[0][i] ) for i in most_sim_idx ] 


In [11]:
by_tf = MostElem( tf_feature_names, term_vector_via_tf, max_n=4  )
by_tfidf = MostElem( tfidf_feature_names, term_vector_via_tfidf, max_n=4  )
by_coword = MostElem( tf_feature_names, term_vector_via_coword, max_n=4  )
by_tscore = MostElem( tf_feature_names, term_vector_via_tscore, max_n=4  )

In [12]:
test_herb = "마황"

print( by_tf.most_similar( test_herb ) )
print( by_tfidf.most_similar( test_herb ) )
print( by_coword.most_similar( test_herb ) )
print( by_tscore.most_similar( test_herb ) )
print( w2v_most_similar( test_herb ) )

[('행인', 0.2784141049539053), ('감초', 0.24777930727725683), ('백지', 0.19616577223915038), ('길경', 0.18640087762763416)]
[('행인', 0.31925387801792854), ('감초', 0.22711443809166312), ('계지', 0.17022316683287525), ('석고', 0.1555594113372934)]
[('감초', 0.7447285115699573), ('길경', 0.7130944511806897), ('백지', 0.7097225397218855), ('행인', 0.6967252582467951)]
[('백지', 0.8559774858181822), ('방풍', 0.8558905512591956), ('길경', 0.8555074442828369), ('감초', 0.8524383084076438)]
[('세신', 0.4089393615722656), ('방풍', 0.401573121547699), ('전호', 0.3826691508293152), ('강활', 0.3817547559738159)]


### Formula Generating Game

In [13]:
def fm_chain( init_herb, most_fn ):
    output = [init_herb]
    next_herb = init_herb
    i = 0
    while i < 20:
        next_herb = most_fn( next_herb )[0][0]
        if next_herb in output:
            break
        output.append( next_herb )
        i = i + 1

    return output

In [14]:
test_herb = "부자"

print(  fm_chain( test_herb, by_tf.most_similar ) )
print(  fm_chain( test_herb, by_tfidf.most_similar ) )
print(  fm_chain( test_herb, by_coword.most_similar ) )
print(  fm_chain( test_herb, by_tscore.most_similar ) )
print(  fm_chain( test_herb, w2v_most_similar ) )


['부자', '건강']
['부자', '건강']
['부자', '건강', '백출', '인삼']
['부자', '육계', '당귀', '천궁']
['부자', '포부자', '교이', '이당']
